# Function Secret Sharing (FSS)
### Function secret sharing is a scheme to share a function $f_s(x): H(x)^s$
The value $s$ is a secret the defines the secret function. It is first shared using Shamir secret sharing. Then, each share holder (for a given $x$) can evaluate a partial result of the function $f_s(x)$. The partial results can be combined to contruct the value of $H(x)^s$.

This demo shows how can we share a secret function and recompute its results in a distributed way. It also shows the homomorphic property: $f_{s_1+s_2}(x) = f_{s_1}(x) \cdot f_{s_2}(x)$. Specifically, by summing the shares of two secret functions, we can evaluate the result of the multiplication of these functions

In [4]:
from ftsa.protocols.buildingblocks.FunctionSS import FSS, FShare
from ftsa.protocols.buildingblocks.JoyeLibert import JLS
from ftsa.protocols.buildingblocks.utils import powmod
import random
from math import factorial

threshold = 70
nclients  = 100
inputsize = 16
dimension = 1
keylength = 1024

JL = JLS(nclients)
pp,_,_ = JL.Setup()

In [5]:
# create two secret functions
s1 = random.SystemRandom().getrandbits(keylength) 
s2 = random.SystemRandom().getrandbits(keylength)
s = s1+s2

# initialize the FSS scheme
FS = FSS(keylength, pp.n)


# share two secrets
allshares1 = FS.share(threshold, nclients, s1)
allshares2 = FS.share(threshold, nclients, s2)

# pick a threshold number of shares randomly 
random.seed(1)
shares1 = random.sample(allshares1, threshold)
random.seed(1)
shares2 = random.sample(allshares2, threshold)

# compute the sum of the picked shares 
shares = [x+y for x,y in zip(shares1, shares2)]

In [6]:

# pick a random t to evaluate the function on
t = random.randint(0,2**16-1)

# evaluate the function on t: f_s(t)  using each share
evalsahres1 = [s.eval(t) for s in shares1]
evalsahres2 = [s.eval(t) for s in shares2]
evalsahres = [s.eval(t) for s in shares]

delta = factorial(nclients)

# reconstruct the results of f_s(t) 
lagcoef = FS.lagrange(shares1, delta)
reconstucted1 =  FS.recon(evalsahres1, delta, lagcoef)
reconstucted2 =  FS.recon(evalsahres2, delta, lagcoef)
reconstucted =  FS.recon(evalsahres, delta, lagcoef)

# check if the reconstructed values match the result of function f_s(t)

print("Verify f_s1(t): ", powmod(FShare.H(t), s1 * delta,pp.nsquare)  == reconstucted1 )
print("Verify f_s2(t): ", powmod(FShare.H(t), s2 * delta,pp.nsquare) == reconstucted2)
print("Verify f_{s1+s2}(t): ", powmod(FShare.H(t), (s1+s2) * delta,pp.nsquare) == reconstucted)


Verify f_s1(t):  True
Verify f_s2(t):  True
Verify f_{s1+s2}(t):  True


### Vector Function Secret Sharing (VFSS)
This is the same as FSS except that we share a function that outputs a vector $g_s(x, v) : [H(x||0)^s, H(x||1)^s, ...,  H(x||v)^s]$ 

In [7]:
vsize = 10

# pick a random t to evaluate the function on
t = random.randint(0,2**16-1)

# evaluate the function on t: f_s(t)  using each share
evalsahresv1 = [s.eval_vector(t, vsize) for s in shares1]
evalsahresv2 = [s.eval_vector(t, vsize) for s in shares2]
evalsahresv = [s.eval_vector(t, vsize) for s in shares]

delta = factorial(nclients)

# reconstruct the results of f_s(t) 
lagcoef = FS.lagrange(shares1, delta)
reconstucted1 =  FS.recon_vector(evalsahresv1, delta, lagcoef)
reconstucted2 =  FS.recon_vector(evalsahresv2, delta, lagcoef)
reconstucted =  FS.recon_vector(evalsahresv, delta, lagcoef)

# check if the reconstructed values match the result of function f_s(t)
valid = True
Y1 = []
Y2 = []
Y = []
for i in range(len(reconstucted1)):
    h = FShare.H((i << keylength // 4) | t)
    Y1.append(powmod(h, s1 * delta,pp.nsquare))
    Y2.append(powmod(h, s2 * delta,pp.nsquare))
    Y.append(powmod(h, s * delta,pp.nsquare))

print("Verify f_s1(t): ", Y1  == reconstucted1 )
print("Verify f_s2(t): ", Y2 == reconstucted2)
print("Verify f_{s1+s2}(t): ", Y == reconstucted)


Verify f_s1(t):  True
Verify f_s2(t):  True
Verify f_{s1+s2}(t):  True
